# Pair Programming ETL II

Tendréis que usar el csv attacks_limpieza_completa que tenéis adjunto abajo.
En la lección de hoy aprendimos como transformar nuestros datos para que estén preparados para almacearlos en una BBDD. En este momento tenemos dos fuentes de datos:
1. El csv con los ataques de tiburones que hemos estado limpiando hasta ahora, el que os hemos adjuntado (attacks_limpieza_completa). Sentiros libres de usar vuestros propios csv en caso de que queráis.
2. El csv con los datos climáticos de los principales paises que tienen ataques de tiburones, el que creamos en el pair programming de ayer.

El objetivo de la sesión de hoy será juntar en un único csv la información de ambas fuentes. Para ello:
- Cargaremos los dos ficheros de datos
- Del dataframe de los ataques nos quedaremos solo con las filas de los países que seleccionamos en la lección de ayer:
    -  USA
    -  Australia
    -  New Zealand
    -  South Africa
    -  Papua New Guinea
- Del dataframe de los datos climáticos seleccionaremos todas las columnas.
- Cuando ya tengamos todos los datos deseados juntaremos los dos csv.
    - Para hacer esta unión tendremos que hacer un groupby en la tabla de clima para sacar una media de las medidas climáticas por país.
    - Antes de hacer el groupby si nos fijamos tenemos dos columnas rh_profile y wind_profile cuya información es una lista de diccionarios. Si intentamos hacer la media de eso no nos dará un valor real. A este problema ya nos enfrentamos en la clase invertida de ETL-2, donde teníais un Bonus para desempaquetar esta información. En caso de que en aquel ejercicio no lo consigierais os dejamos por aquí una posible solución que nos permite separar esa información en distintas columnas. Os dejamos el código documentado. ⚠️ Os recomendamos que vayáis desgranando el código y viendo lo que nos devuelve cada línea de código para entenderlo mejor.

In [1]:
from IPython.core.interactiveshell import InteractiveShell 
InteractiveShell.ast_node_interactivity = "all"

import requests
import pandas as pd
import numpy as np



In [2]:
pd.options.display.max_columns = None

In [3]:
df_attack = pd.read_csv("../datos/datos_limpiezaV.csv", index_col=0)
df_clima = pd.read_csv("../datos/datos_clima.csv",index_col=0)

In [4]:
df_clima.head()

,timepoint,cloudcover,highcloud,midcloud,lowcloud,rh_profile,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,country
0,3,9,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 12}, {'layer': '900m...","[{'layer': '950mb', 'direction': 180, 'speed':...",27,-4,11,-9999,none,1,0,205,2,usa
1,6,9,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 12}, {'layer': '900m...","[{'layer': '950mb', 'direction': 275, 'speed':...",27,-4,11,-9999,rain,1,0,240,2,usa
2,9,9,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 11}, {'layer': '900m...","[{'layer': '950mb', 'direction': 195, 'speed':...",27,-1,10,-9999,rain,1,0,265,2,usa
3,12,7,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 11}, {'layer': '900m...","[{'layer': '950mb', 'direction': 195, 'speed':...",27,-1,10,-9999,rain,1,0,200,2,usa
4,15,5,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 13}, {'layer': '900m...","[{'layer': '950mb', 'direction': 195, 'speed':...",27,-1,10,-9999,none,1,0,195,3,usa


In [5]:
df_attack.head()

,year,country,age,species,date,month,fatal,sex,main_species,age_numbers,age_numbers_mean,sex_moda
0,2018.0,usa,57,White shark,25-Jun-2018,Jun,n,F,white_shark,57.0,57.0,F
1,2018.0,brazil,18,Tiger shark,03-Jun-2018,Jun,y,M,tiger_shark,18.0,18.0,M
2,2018.0,usa,15,"Bull shark, 6'",26-May-2018,May,n,M,bull_shark,15.0,15.0,M
3,2018.0,australia,32,Grey reef shark,24-May-2018,May,n,M,grey_shark,32.0,32.0,M
4,2018.0,england,21,Invalid incident,13-May-2018,May,n,M,unspecified,21.0,21.0,M


In [6]:
attaques = df_attack[(df_attack["country"] == "usa") | (df_attack["country"] == "australia") | (df_attack["country"] == "papua new guinea") | (df_attack["country"] == "new zealand")| (df_attack["country"] == "south africa")]
attaques.head()
                     

,year,country,age,species,date,month,fatal,sex,main_species,age_numbers,age_numbers_mean,sex_moda
0,2018.0,usa,57,White shark,25-Jun-2018,Jun,n,F,white_shark,57.0,57.0,F
2,2018.0,usa,15,"Bull shark, 6'",26-May-2018,May,n,M,bull_shark,15.0,15.0,M
3,2018.0,australia,32,Grey reef shark,24-May-2018,May,n,M,grey_shark,32.0,32.0,M
6,2018.0,australia,60,3 m shark,25-Apr-2018,Apr,n,M,unspecified,60.0,60.0,M
8,2018.0,south africa,33,"White shark, 2.5 m",22-Apr-2018,Apr,n,M,white_shark,33.0,33.0,M


In [7]:
df_clima.reset_index()

,index,timepoint,cloudcover,highcloud,midcloud,lowcloud,rh_profile,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,country
0,0,3,9,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 12}, {'layer': '900m...","[{'layer': '950mb', 'direction': 180, 'speed':...",27,-4,11,-9999,none,1,0,205,2,usa
1,1,6,9,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 12}, {'layer': '900m...","[{'layer': '950mb', 'direction': 275, 'speed':...",27,-4,11,-9999,rain,1,0,240,2,usa
2,2,9,9,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 11}, {'layer': '900m...","[{'layer': '950mb', 'direction': 195, 'speed':...",27,-1,10,-9999,rain,1,0,265,2,usa
3,3,12,7,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 11}, {'layer': '900m...","[{'layer': '950mb', 'direction': 195, 'speed':...",27,-1,10,-9999,rain,1,0,200,2,usa
4,4,15,5,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 13}, {'layer': '900m...","[{'layer': '950mb', 'direction': 195, 'speed':...",27,-1,10,-9999,none,1,0,195,3,usa
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315,59,180,9,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 14}, {'layer': '900m...","[{'layer': '950mb', 'direction': 195, 'speed':...",27,-1,11,-9999,none,3,0,195,3,papua new guinea
316,60,183,7,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 14}, {'layer': '900m...","[{'layer': '950mb', 'direction': 185, 'speed':...",27,-1,11,-9999,none,3,0,185,3,papua new guinea
317,61,186,7,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 14}, {'layer': '900m...","[{'layer': '950mb', 'direction': 170, 'speed':...",27,-1,10,-9999,none,3,0,170,2,papua new guinea
318,62,189,7,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 14}, {'layer': '900m...","[{'layer': '950mb', 'direction': 160, 'speed':...",27,-1,10,-9999,none,3,0,175,2,papua new guinea


In [8]:

import ast

df_clima['wind_profile']= df_clima['wind_profile'].apply(ast.literal_eval)

# una vez que tengamos la columna cambiada, una fantasía de Pandas es que si hago un apply sobre una columna cuyos valores son diccionarios o listas nos va a genererar una columna con los valores de los diccionarios o listas. Donde cada columna será key del diccionario o cada elemento de la lista. 


x = df_clima['wind_profile'].apply(pd.Series)


# nos creamos un dataframe nuevo con el resultado de la información de una de las columnas separadas por columnas. Esto nos va a devolver un dataframe donde cada fila será una celda del dataframe anterior. 
x = df_clima['rh_profile'].apply(pd.Series) 

# ¿Qué es lo que ocurre cuando hacemos esto?
# Nos ha creado tantas columnas como valores tuvieramos en la lista. Donde columna es, en este caso, un diccionario (porque nuestra lista esta compuesta por distintos diccionarios)

# Ok, hemos conseguido desempaquetar la información de la lista en distintas columnas. Ahora tenemos que despempaquetar la información de los diccionarios en distintas columnas. En este caso, lo que querremos es que las key de los diccionarios sean los nombres de las columnas y los values los valores de las celdas del dataframe. Volveremos a seguir entonces la misma lógica que antes con el apply, pero en este caso necesitamos hacerlo para todo el dataframe (que es x): 

# Por eso empezamos con un for para iterar por cada una de las columnas. 
for i in range(len(x.columns)): 

    # aplicamos el apply,extraemos el valore de la key "layer" y lo almacenamos en una variable que convertimos a string 
    nombre = "rh_" + str(x[i].apply(pd.Series)["layer"][0]) 

    # hacemos lo mismo con una variable que se llame valores para "guardar" los valores de la celda
    valores = list(x[i].apply(pd.Series)["rh"] )

    # usamos el método insert de los dataframes para ir añadiendo esta información a el dataframe con la información del clima. 
    df_clima.insert(i, nombre, valores)

# una vez que hayamos hecho esto para las dos columnas ya podremos hacer el gropuby para después unir toda la información. 

KeyError: 'layer'

In [ ]:
rh[0].apply(pd.Series)
#nombre de la columna es layer que es la key de la columna 0
#mientras la columna rh son los valores de la key rh 

: 

: 

In [ ]:
rh.head(1)

for i in range(len(rh.columns)): 
    rh[i].apply(pd.Series)
    #print (x)

    # aplicamos el apply,extraemos el valore de la key "layer" y lo almacenamos en una variable que convertimos a string 
    nombre = "rh_" + str(rh[i].apply(pd.Series)["layer"][0]) 
    print(nombre)
    # hacemos lo mismo con una variable que se llame valores para "guardar" los valores de la celda
    valores = list(rh[i].apply(pd.Series)["rh"] )

    # usamos el método insert de los dataframes para ir añadiendo esta información a el dataframe con la información del clima. 
    df_clima.insert(i, nombre, valores)
    



: 

: 

: 

In [ ]:
df_clima.head()

: 

: 